In [1]:
pip install pygame

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pygame
import random
import math
import colorsys
from pygame.locals import *

pygame 2.6.0 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
idx = [0, 0, 0]

def mark_pixel(surface, pos, pcol):
    col = surface.get_at(pos)
    surface.set_at(pos, (min(col[0] + pcol[0]/10, 255),
                         min(col[1] + pcol[1]/10, 255),
                         min(col[2] + pcol[2]/10, 255)))

def random_point_index(p):
    if len(p) <= 3:
        return random.randint(0, len(p) - 1)

    global idx
    idx[2] = idx[1]
    idx[1] = idx[0]
    dst1 = abs(idx[1] - idx[2])

    while True:
        idx[0] = random.randint(0, len(p) - 1)
        dst = abs(idx[0] - idx[1])
        if dst1 == 0 and (dst == 1 or dst == len(p) - 1):
            continue
        else:
            break

    return idx[0]

def init_polygon(width, height, n):
    delta_angle = 360/n
    r = width/2 - 10
    p = []

    for i in range(0, n):
        angle = (180 + i*delta_angle) * math.pi / 180
        color = colorsys.hsv_to_rgb((i*delta_angle)/360, 0.8, 1)
        p.append(((width/2 + r*math.sin(angle),
                   height/2 + r*math.cos(angle)),
                  (int(color[0]*255), int(color[1]*255), int(color[2]*255))))

    return p

def main(width, height, n, r):
    pygame.init()
    surface = pygame.display.set_mode((width, height))
    pygame.display.set_caption('Das Chaos Spiel')

    p = init_polygon(width, height, n)

    x, y = (400, 300)
    step = 0
    while True:
        step = step + 1
        point_idx = random_point_index(p)

        pos = p[point_idx][0]
        color = p[point_idx][1]
        x += (pos[0] - x) * r
        y += (pos[1] - y) * r

        mark_pixel(surface, (int(x), int(y)), color)

        if step % 1000 == 0:
            pygame.display.update()

        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.image.save(surface, 'chaosspiel.jpg')
                pygame.quit()
                return

if __name__ == "__main__":
    n=5; main(800, 800, n,  0.45)

In [4]:
import pygame

# Initialize Pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Mandelbrot Set")

# Mandelbrot parameters
MAX_ITER = 50  # Maximum iterations for each point
zoom = 1.0
offset_x = 0.0
offset_y = 0.0

def mandelbrot(c):
    z = 0
    n = 0
    while abs(z) <= 2 and n < MAX_ITER:
        z = z*z + c
        n += 1
    return n

def draw_mandelbrot():
    for x in range(WIDTH):
        for y in range(HEIGHT):
            # Map pixel coordinates to complex plane
            real = (x / WIDTH) * 3.5 / zoom - (2.5 / zoom) + offset_x
            imag = (y / HEIGHT) * 2.5 / zoom - (1.25 / zoom) + offset_y
            c = complex(real, imag)

            iterations = mandelbrot(c)

            # Color mapping
            if iterations == MAX_ITER:
                color = (0, 0, 0)  # Black for points in the set
            else:
                # Simple color gradient based on iteration count
                color = (iterations * 5 % 255, iterations * 7 % 255, iterations * 10 % 255)
            
            screen.set_at((x, y), color)
    pygame.display.flip()

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            # Zoom in on mouse click
            mouse_x, mouse_y = event.pos
            
            # Calculate new offset based on mouse click
            new_real = (mouse_x / WIDTH) * 3.5 / zoom - (2.5 / zoom) + offset_x
            new_imag = (mouse_y / HEIGHT) * 2.5 / zoom - (1.25 / zoom) + offset_y
            
            zoom *= 1.5  # Increase zoom level
            offset_x += (new_real - ( (mouse_x / WIDTH) * 3.5 / zoom - (2.5 / zoom) ) )
            offset_y += (new_imag - ( (mouse_y / HEIGHT) * 2.5 / zoom - (1.25 / zoom) ) )
            
            draw_mandelbrot() # Recalculate and redraw

    draw_mandelbrot() # Initial draw

pygame.quit()